In [27]:
from geopy.geocoders import Nominatim
import time
import matplotlib.pyplot as plt
from rasterio.plot import show
from rasterio.enums import Resampling
from pyproj import Transformer
import rasterio
import netCDF4 as nc
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
from collections import defaultdict
from flask import Flask, request, jsonify
import io
import base64
import statistics
import concurrent.futures

In [29]:
file_path_ar6_ssp585 = 'ar6data/total_ssp585_medium_confidence_values.nc'

file_path_ar6_ssp245 = 'ar6data/total_ssp245_medium_confidence_values.nc'

In [31]:
storm_and_flood_csv = "cmip6data/StormEvents_details-ftp_v1.0_d2000_c20220425.csv"
# Define paths to the inundation geotiffs
inundation_maps_MA = {
    0: "inundationmaps/MA_connectRaster_0.tif",
    0.5: "inundationmaps/MA_connectRaster_0_5.tif",
    1: "inundationmaps/MA_connectRaster_1.tif",
    1.5: "inundationmaps/MA_connectRaster_1_5.tif",
    2: "inundationmaps/MA_connectRaster_2.tif",
    2.5: "inundationmaps/MA_connectRaster_2_5.tif",
    3: "inundationmaps/MA_connectRaster_3.tif",
    3.5: "inundationmaps/MA_connectRaster_3_5.tif",
    4: "inundationmaps/MA_connectRaster_4.tif"
}

inundation_maps_PA = {
    0: "inundationmaps/PA_connectRaster_0.tif",
    0.5: "inundationmaps/PA_connectRaster_0_5.tif",
    1: "inundationmaps/PA_connectRaster_1.tif",
    1.5: "inundationmaps/PA_connectRaster_1_5.tif",
    2: "inundationmaps/PA_connectRaster_2.tif",
    2.5: "inundationmaps/PA_connectRaster_2_5.tif",
    3: "inundationmaps/PA_connectRaster_3.tif",
    3.5: "inundationmaps/PA_connectRaster_3_5.tif",
    4: "inundationmaps/PA_connectRaster_4.tif"
}

inundation_maps_RI = {
    0: "inundationmaps/RI_connectRaster_0.tif",
    0.5: "inundationmaps/RI_connectRaster_0_5.tif",
    1: "inundationmaps/RI_connectRaster_1.tif",
    1.5: "inundationmaps/RI_connectRaster_1_5.tif",
    2: "inundationmaps/RI_connectRaster_2.tif",
    2.5: "inundationmaps/RI_connectRaster_2_5.tif",
    3: "inundationmaps/RI_connectRaster_3.tif",
    3.5: "inundationmaps/RI_connectRaster_3_5.tif",
    4: "inundationmaps/RI_connectRaster_4.tif"
}

inundation_maps_AL = {
    0: "inundationmaps/AL_connectRaster_0.tif",
    1: "inundationmaps/AL_connectRaster_1.tif",
    1.5: "inundationmaps/AL_connectRaster_1_5.tif",
    2: "inundationmaps/AL_connectRaster_2.tif",
    2.5: "inundationmaps/AL_connectRaster_2_5.tif",
    3: "inundationmaps/AL_connectRaster_3.tif",
    3.5: "inundationmaps/AL_connectRaster_3_5.tif",
    4: "inundationmaps/AL_connectRaster_4.tif"
}

inundation_maps_CT = {
    0: "inundationmaps/CT_connectRaster_0.tif",
    0.5: "inundationmaps/CT_connectRaster_0_5.tif",
    1: "inundationmaps/CT_connectRaster_1.tif",
    1.5: "inundationmaps/CT_connectRaster_1_5.tif",
    2: "inundationmaps/CT_connectRaster_2.tif",
    2.5: "inundationmaps/CT_connectRaster_2_5.tif",
    3: "inundationmaps/CT_connectRaster_3.tif",
    3.5: "inundationmaps/CT_connectRaster_3_5.tif",
    4: "inundationmaps/CT_connectRaster_4.tif"
}

inundation_maps_DC = {
    0: "inundationmaps/DC_connectRaster_0.tif",
    0.5: "inundationmaps/DC_connectRaster_0_5.tif",
    1: "inundationmaps/DC_connectRaster_1.tif",
    1.5: "inundationmaps/DC_connectRaster_1_5.tif",
    2: "inundationmaps/DC_connectRaster_2.tif",
    2.5: "inundationmaps/DC_connectRaster_2_5.tif",
    3: "inundationmaps/DC_connectRaster_3.tif",
    3.5: "inundationmaps/DC_connectRaster_3_5.tif",
    4: "inundationmaps/DC_connectRaster_4.tif"
}

inundation_maps_DE = {
    0: "inundationmaps/DE_connectRaster_0.tif",
    0.5: "inundationmaps/DE_connectRaster_0_5.tif",
    1: "inundationmaps/DE_connectRaster_1.tif",
    1.5: "inundationmaps/DE_connectRaster_1_5.tif",
    2: "inundationmaps/DE_connectRaster_2.tif",
    2.5: "inundationmaps/DE_connectRaster_2_5.tif",
    3: "inundationmaps/DE_connectRaster_3.tif",
    3.5: "inundationmaps/DE_connectRaster_3_5.tif",
    4: "inundationmaps/DE_connectRaster_4.tif"
}

inundation_maps_MS = {
    0: "inundationmaps/MS_connectRaster_0.tif",
    0.5: "inundationmaps/MS_connectRaster_0_5.tif",
    1: "inundationmaps/MS_connectRaster_1.tif",
    1.5: "inundationmaps/MS_connectRaster_1_5.tif",
    2: "inundationmaps/MS_connectRaster_2.tif",
    2.5: "inundationmaps/MS_connectRaster_2_5.tif",
    3: "inundationmaps/MS_connectRaster_3.tif",
    3.5: "inundationmaps/MS_connectRaster_3_5.tif",
    4: "inundationmaps/MS_connectRaster_4.tif"
}

inundation_maps_NH = {
    0: "inundationmaps/NH_connectRaster_0.tif",
    0.5: "inundationmaps/NH_connectRaster_0_5.tif",
    1: "inundationmaps/NH_connectRaster_1.tif",
    1.5: "inundationmaps/NH_connectRaster_1_5.tif",
    2: "inundationmaps/NH_connectRaster_2.tif",
    2.5: "inundationmaps/NH_connectRaster_2_5.tif",
    3: "inundationmaps/NH_connectRaster_3.tif",
    3.5: "inundationmaps/NH_connectRaster_3_5.tif",
    4: "inundationmaps/NH_connectRaster_4.tif"
}

inundation_maps_FL = {
    0: "inundationmaps/FL_SE_connectRaster_0_0.tif",
    0.5: "inundationmaps/FL_SE_connectRaster_0_5.tif",
    1: "inundationmaps/FL_SE_connectRaster_1_0.tif",
    1.5: "inundationmaps/FL_SE_connectRaster_1_5.tif",
    2: "inundationmaps/FL_SE_connectRaster_2_0.tif",
    2.5: "inundationmaps/FL_SE_connectRaster_2_5.tif",
    3: "inundationmaps/FL_SE_connectRaster_3_0.tif",
    3.5: "inundationmaps/FL_SE_connectRaster_3_5.tif",
    4: "inundationmaps/FL_SE_connectRaster_4_0.tif"
}

inundation_maps_CA = {
    0: "inundationmaps/CA_LOX_connectRaster_0.tif",
    1: "inundationmaps/CA_LOX_connectRaster_1.tif",
    2: "inundationmaps/CA_LOX_connectRaster_2.tif",
    3: "inundationmaps/CA_LOX_connectRaster_3.tif",
    4: "inundationmaps/CA_LOX_connectRaster_4.tif"
}

inundation_maps_NY = {
    0: "inundationmaps/NY_M_connectRaster_0.tif",
    0.5: "inundationmaps/NY_M_connectRaster_0_5.tif",
    1: "inundationmaps/NY_M_connectRaster_1.tif",
    1.5: "inundationmaps/NY_M_connectRaster_1_5.tif",
    2: "inundationmaps/NY_M_connectRaster_2.tif",
    2.5: "inundationmaps/NY_M_connectRaster_2_5.tif",
    3: "inundationmaps/NY_M_connectRaster_3.tif",
    3.5: "inundationmaps/NY_M_connectRaster_3_5.tif",
    4: "inundationmaps/NY_M_connectRaster_4.tif"
}

state_inundation_maps = {
    "Massachusetts": inundation_maps_MA,
    "MA": inundation_maps_MA,
    "Pennsylvania": inundation_maps_PA,
    "PA": inundation_maps_PA,
    "Rhode Island": inundation_maps_RI,
    "RI": inundation_maps_RI,
   "Alabama": inundation_maps_AL,
   "AL": inundation_maps_AL,
    "Connecticut": inundation_maps_CT,
    "CT": inundation_maps_CT,
    "Washington, DC": inundation_maps_DC,
    "DC": inundation_maps_DC,
    "Delaware": inundation_maps_DE,
    "DE": inundation_maps_DE,
    "Mississippi": inundation_maps_MS,
    "MS": inundation_maps_MS,
    "New Hampshire": inundation_maps_NH,
    "NH": inundation_maps_NH,
    "FL": inundation_maps_FL,
    "Florida": inundation_maps_FL,
    "California": inundation_maps_CA,
    "CA": inundation_maps_CA,
    "New York": inundation_maps_NY,
    "NY": inundation_maps_NY
}

In [33]:
def get_sea_level_projection(input_lat, input_lon, file_path):
    # Load the dataset
    ds = nc.Dataset(file_path, 'r')
    
    # Extract latitude, longitude, sea level change, and years
    latitude = ds.variables['lat'][:]
    longitude = ds.variables['lon'][:]
    sea_level_change = ds.variables['sea_level_change'][:]  # (quantiles, years, locations)
    years = ds.variables['years'][:]
    
    # Adjust longitude to [-180, 180] if necessary
    if np.max(longitude) > 180:
        longitude = np.where(longitude > 180, longitude - 360, longitude)
    
    # Function to find nearby points within a given threshold
    def find_nearby_points(lat_threshold, lon_threshold, max_points=5):
        nearby_lat = np.argwhere(np.abs(latitude - input_lat) < lat_threshold).flatten()
        nearby_lon = np.argwhere(np.abs(longitude[nearby_lat] - input_lon) < lon_threshold).flatten()
        qualified_idx = nearby_lat[nearby_lon]
        return qualified_idx[:max_points]  # Return at most max_points
    
    # Try finding points within 0.5 degrees
    qualified_idx = find_nearby_points(0.5, 0.5)
    
    # If no points found, expand the search to 1 degree
    if len(qualified_idx) == 0:
        qualified_idx = find_nearby_points(1.0, 1.0)
    
    # If still no points found, return an empty DataFrame
    if len(qualified_idx) == 0:
        return None
    
    # Step 2: Extract sea level change data for the nearby coordinates
    # Assuming quantile 50 (median) for sea level rise projections
    quantile_idx = 50  # Adjust if necessary

    sea_level_rise_data = []
    for idx in qualified_idx:
        slr_at_location = sea_level_change[quantile_idx, :, idx]  # Sea level rise at this location for all years
        sea_level_rise_data.append(slr_at_location)

    # Step 3: Average sea level rise over all nearby locations
    sea_level_rise_avg = np.mean(sea_level_rise_data, axis=0)

    # Create a DataFrame for the time series
    df = pd.DataFrame({
        'Year': years,
        'Sea Level Rise': sea_level_rise_avg
    })
    df = df[df['Year'] <= 2100]
    # Close the dataset
    ds.close()
    if not np.ma.is_masked(df):
        return df
    return None

In [35]:
def get_map_boundary(inundation_map):
    with rasterio.open(inundation_map) as src:
            # Check the bounds of the raster file (lon_min, lon_max should be in [-180, 180])
            lat_min, lat_max = src.bounds.bottom, src.bounds.top
            lon_min, lon_max = src.bounds.left, src.bounds.right
    return lat_min, lat_max, lon_min, lon_max

In [37]:
# Function to check if a coordinate is inundated using a smaller window
def check_inundation(file_path, lat, lon, window_size=5):
    def check_point(lat, lon):
        """Helper function to check inundation for a single point."""
        with rasterio.open(file_path) as src:
            # Check the bounds of the raster file (lon_min, lon_max should be in [-180, 180])
            lat_min, lat_max = src.bounds.bottom, src.bounds.top
            lon_min, lon_max = src.bounds.left, src.bounds.right
            
            # Transform the input coordinates to the raster's CRS
            transformer = Transformer.from_crs("EPSG:4326", src.crs, always_xy=True)
            x, y = transformer.transform(lon, lat)
            
            # Check if the input coordinate is within the bounds of the raster
            if not (lon_min <= x <= lon_max and lat_min <= y <= lat_max):
                return "OUTSIDE BOUNDARY"
            
            # Get the row and column indices for the input coordinate
            row, col = src.index(x, y)
            
            # Define a window around the input coordinate
            row_start = max(row - window_size, 0)
            row_end = min(row + window_size, src.height)
            col_start = max(col - window_size, 0)
            col_end = min(col + window_size, src.width)
            
            window = rasterio.windows.Window(col_start, row_start, col_end - col_start, row_end - row_start)
            
            # Read only the window of interest and ensure it's valid
            try:
                data = src.read(1, window=window, masked=True)
            except Exception as e:
                print(f"Error reading data in the window: {e}")
                return "OUTSIDE BOUNDARY"

            # Calculate local row and column within the window
            local_row = row - row_start
            local_col = col - col_start

            # Ensure the indices are within bounds of the data window
            if local_row >= data.shape[0] or local_col >= data.shape[1]:
                return "OUTSIDE BOUNDARY"
            
            # Check the value at the location and handle NoData explicitly
            value_at_location = data[local_row, local_col]

            # Handle NoData and inundation values explicitly
            if value_at_location == 0 or value_at_location == 0.0:
                return False  # Uninundated
            elif value_at_location == 1 or value_at_location == 1.0:
                return True  # Inundated
            else:
                return None

    # First check the original input point
    result = check_point(lat, lon)
    if result in [True, False]:  # If we get a valid result (inundated or not), return it
        #print("The value is taken directly from the input coordinate (not from surrounding buffer)")
        return result
    elif result == "OUTSIDE BOUNDARY":
        return "OUTSIDE BOUNDARY"

    # If the original point has no value, check surrounding 8 points
    lat_offset = 0.0001
    lon_offset = 0.0001
    nearby_points = [
        (lat + lat_offset, lon),
        (lat - lat_offset, lon),
        (lat, lon + lon_offset),
        (lat, lon - lon_offset),
        (lat + lat_offset, lon + lon_offset),
        (lat - lat_offset, lon - lon_offset),
        (lat + lat_offset, lon - lon_offset),
        (lat - lat_offset, lon + lon_offset),
    ]
    nearby_results = []
    for nearby_lat, nearby_lon in nearby_points:
        nearby_result = check_point(nearby_lat, nearby_lon)
        if nearby_result == True:  # Prioritize inundation
            #print("The value is taken from surrounding buffer")
            return True
        elif nearby_result == False:  # Uninundated
            #print("The value is taken from surrounding buffer")
            nearby_results.append(nearby_result)
            continue  # Keep checking for inundation
        # Skip if no value (None or "UNKNOWN_VALUE or NO_DATA")
    if nearby_results:
        return False
    # If none of the points had a value, return no value
    return None

In [39]:
# Initialize the geolocator with a more specific user agent
geolocator = Nominatim(user_agent="my_geocoding_application")

def get_place_name(lat, lon):
    try:
        # Introduce a delay to avoid rate limiting
        time.sleep(0.5)  # Wait for 1 second between requests
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and location.address:
            address = location.raw['address']
            # Return the first available place name: city, town, or village
            if 'city' in address:
                return address['city']
            elif 'town' in address:
                return address['town']
            elif 'village' in address:
                return address['village']
            else:
                return "Unknown"
        else:
            return "Unknown"
    except Exception as e:
        return "Unknown"

In [41]:
def find_closest_uninundated_locations(input_lat, input_lon, inundation_map, search_radius=0.08, step_size=0.02, max_locations=3, safe_margin=0.005, safe_step=0.0025):
    # Get the boundary of the map (latitude and longitude limits)
    
    lat_min, lat_max, lon_min, lon_max = get_map_boundary(inundation_map)
    if not (lon_min <= input_lon <= lon_max and lat_min <= input_lat <= lat_max):
        return None
    # List to store safe locations
    safe_locations = []
    unique_place_names = set()  # Set to avoid duplicate place names

    layer = 1  # Start at layer 1 (surrounding the input point)
    
    # Continue searching until the required number of safe locations is found or search_radius is exceeded
    while len(safe_locations) < max_locations and layer * step_size <= search_radius:
        # Generate the nearby points for the current layer
        nearby_points = []
        
        # Top row (moving right)
        for i in range(-layer, layer + 1):
            nearby_points.append((input_lat + i * step_size, input_lon + layer * step_size))
        
        # Right column (moving down)
        for i in range(layer - 1, -layer - 1, -1):
            nearby_points.append((input_lat + layer * step_size, input_lon + i * step_size))
        
        # Bottom row (moving left)
        for i in range(layer - 1, -layer - 1, -1):
            nearby_points.append((input_lat + i * step_size, input_lon - layer * step_size))
        
        # Left column (moving up)
        for i in range(-layer + 1, layer):
            nearby_points.append((input_lat - layer * step_size, input_lon + i * step_size))
        
        # Filter out points outside the map boundaries
        nearby_points = [(lat, lon) for lat, lon in nearby_points if lon_min <= lon <= lon_max and lat_min <= lat <= lat_max]

        # Check the nearby points for inundation status and margin safety
        for nearby_lat, nearby_lon in nearby_points:
            # Skip if the point is inundated
            if check_inundation(inundation_map, nearby_lat, nearby_lon):
                continue
            
            # Perform reverse geocoding to get the nearest place name
            place_name = get_place_name(nearby_lat, nearby_lon)
            if place_name not in unique_place_names and place_name != "Unknown":
                # Add the safe location to the list
                safe_locations.append((nearby_lat, nearby_lon, place_name))
                unique_place_names.add(place_name)

            # If we've found the required number of locations, return them
            if len(safe_locations) >= max_locations:
                return safe_locations
        
        # Move to the next outer layer
        layer += 1
    
    # Return the found locations or None if no safe locations are found
    return safe_locations if safe_locations else None

In [43]:
def process_scenario(scenario_name, file_path, inundation_map, input_lat, input_lon):
    coordinates = (input_lat, input_lon)
    bound_check = check_inundation(inundation_map[0], input_lat, input_lon, window_size=10)
    output = ""
    if bound_check == "OUTSIDE BOUNDARY":
        output, ssh_change_df, highest_threshold_exceeded, next_output, inundation_occurred, inundation_year, coordinates, place_name, next_inundation_status = None, None, None, None, None, None, None, None, None
        return output, ssh_change_df, highest_threshold_exceeded, next_output, inundation_occurred, inundation_year, coordinates, place_name, next_inundation_status
        
    slr_df = get_sea_level_projection(input_lat, input_lon, file_path)
    
    # Convert SSH_Change from meters to feet
    mm_to_meters = 0.001
    meters_to_feet = 3.28084
    slr_df['Sea Level Rise'] = slr_df['Sea Level Rise'] * mm_to_meters * meters_to_feet

    # Track thresholds exceeded, their corresponding years, and whether inundation occurs
    threshold_exceedance = []

    available_thresholds = sorted(inundation_map.keys())
    highest_threshold_exceeded = 0
    inundation_occurred = False
    inundation_year = None
    for threshold in available_thresholds:
        exceedance_year = slr_df[slr_df['Sea Level Rise'] > threshold]['Year'].min()
        if not pd.isna(exceedance_year):
            file_path = inundation_map[threshold]
            inundation_status = check_inundation(file_path, input_lat, input_lon, window_size=10)  # Pass window_size
            threshold_exceedance.append((threshold, exceedance_year, inundation_status))
            if inundation_status:
                inundation_occurred = True
                highest_threshold_exceeded = threshold
                inundation_year = exceedance_year
                break
            highest_threshold_exceeded = threshold
            
    # Check the next threshold if applicable
    if highest_threshold_exceeded < max(available_thresholds):
        next_threshold_idx = available_thresholds.index(highest_threshold_exceeded) + 1
        if next_threshold_idx < len(available_thresholds):
            next_threshold = available_thresholds[next_threshold_idx]
            next_file_path = inundation_map[next_threshold]
            next_threshold_inundation = check_inundation(next_file_path, input_lat, input_lon, window_size=10)
            next_inundation_status = "Yes" if next_threshold_inundation else "No"
            next_output = f"Next Unmet Threshold: {next_threshold} feet, Year Exceeded: Unknown, Inundation: {next_inundation_status}"
        else:
            next_output = "No higher threshold available to check."

    place_name = get_place_name(input_lat, input_lon)
    if threshold_exceedance:
        for threshold, year, inundated in threshold_exceedance:
            inundation_status = "Yes" if inundated else "No"
            output += f"Threshold: {threshold} feet, Year Exceeded: {year}, Inundation: {inundation_status}"

    return output, slr_df, highest_threshold_exceeded, next_output, inundation_occurred, inundation_year, coordinates, place_name, next_inundation_status

In [45]:
def find_safe_locations(input_lat, input_lon, inundation_map, inundation_occurance, highest_threshold_exceeded, find_radius=0.05, find_step=0.01, max_location=3, max_coord=5, whether_find_loc=True, whether_find_coord=False):
    if inundation_occurance is None:
        return None, None, None, None
    nearby_safe_locations = None
    nearby_safe_coordinates = None
    min_safe_distance = None
    safe_distance = None
    safe_location = ""
    safe_distances = []
    if whether_find_loc:
        nearby_safe_locations = find_closest_uninundated_locations(input_lat, input_lon, inundation_map[highest_threshold_exceeded+1], search_radius=find_radius, step_size=find_step, max_locations=max_location)
        if nearby_safe_locations is not None:
            for lat, lon, place in nearby_safe_locations:
                if lat and lon:
                    safe_distance = geodesic((input_lat, input_lon), (lat, lon)).miles
                    safe_distances.append(safe_distance)
                    safe_location += f"{place} (coordinate: {lat}, {lon}). Distance: {safe_distance:.2f} miles."
                else:
                    safe_location += "No safe location found within the search radius."
            if safe_distances:
                min_safe_distance = np.min(safe_distances)
    return safe_location, nearby_safe_locations, nearby_safe_coordinates, min_safe_distance

In [47]:
weights = {
    'slr': 0.05, #Maximum SSH of the ocean near the location
    'inundation': 0.5, #1, 0.5, 0, whether the inundation will be inundated
    'proximity_safe_location': 0.1, #distance between the closest safe neighborhood and your house. Change it to only when knowing that the house is inundated, only when inundation score yields 0.5 or 0.
    'storm': 0.20, #storm frequency
    'flood': 0.15 #historical flood count
}

def calculate_risk_score(slr_score, inundation_score, proximity_score, storm_score, flood_score, weights):
    # Combine the normalized risks using weights
    if not slr_score:
        total_score = "Not enough data"
        return total_score
    total_score = (
        weights['slr'] * slr_score +
        weights['inundation'] * inundation_score +
        weights['proximity_safe_location'] * proximity_score +
        weights['storm'] * storm_score +
        weights['flood'] * flood_score
    )
    # Scale to 0-100
    total_score *= 100
    return total_score

def calculate_slr_score(max_ssh_change, max_expected_slr=3.5):
    if not max_ssh_change:
        slr_score = None
        return slr_score
    # Normalize SLR (ensure it doesn't exceed max_expected_slr)
    normalized_slr = min(max_ssh_change / max_expected_slr, 1)
    # Invert to make higher values safer
    slr_score = 1 - normalized_slr
    return slr_score #Change it

def calculate_inundation_score(inundation_year):
    if inundation_year is None:
        # Not inundated by 2100
        return 1
    elif inundation_year <= 2030:
        # Inundated before or in 2050
        return 0
    elif inundation_year <= 2040:
        return 0.125
    elif inundation_year <= 2050:
        return 0.25
    elif inundation_year <= 2060:
        return 0.375
    elif inundation_year <= 2070:
        return 0.5
    elif inundation_year <= 2080:
        return 0.625
    elif inundation_year <= 2090:
        return 0.75
    elif inundation_year <= 2100:
        return 0.875
    else:
        # Inundation year beyond 2100 (unlikely in your dataset)
        return 1

def calculate_proximity_score(min_safe_distance, inundation_year, max_distance=2):
    if not min_safe_distance:
        min_safe_distance = 2
    if inundation_year:
        # Normalize distance (ensure it doesn't exceed max_distance)
        normalized_distance = min(min_safe_distance / max_distance, 1)
        # Invert to make closer distances safer
        proximity_score = 1 - normalized_distance
        return proximity_score
    else:
        return 1
        
def get_county_and_state(lat, lon):
    try:
        time.sleep(1)
        location = geolocator.reverse((lat, lon), exactly_one=True)
        if location and location.address:
            address = location.raw['address']
            county_name = address.get('county', 'Unknown')
            state_name = address.get('state', 'Unknown')
            return county_name, state_name
        else:
            return "Unknown", "Unknown"
    except Exception as e:
        return "Unknown", "Unknown"

def normalize_county_name(county_name):
    # Remove common suffixes
    suffixes = ['County', 'Parish', 'Municipality', 'Borough', 'Census Area', 'City and Borough']
    for suffix in suffixes:
        if suffix in county_name:
            county_name = county_name.replace(suffix, '')
    # Strip whitespace and convert to uppercase
    county_name = county_name.strip().upper()
    return county_name

usecols = [
    'BEGIN_LAT', 'BEGIN_LON', 'EVENT_TYPE', 'BEGIN_DATE_TIME',
    'EVENT_ID', 'CZ_NAME', 'CZ_TYPE', 'STATE'
]
storm_and_flood_df = pd.read_csv(storm_and_flood_csv, usecols=usecols)
def get_storm_and_flood_frequency(lat, lon, df, radius_km=20):


    # Filter for storm-related events
    storm_types = ['Thunderstorm Wind', 'Tornado', 'Hail', 'Hurricane']
    storm_events = df[df['EVENT_TYPE'].isin(storm_types)].copy()

    # Filter for flood-related events
    flood_types = ['Flood', 'Flash Flood', 'Coastal Flood']
    flood_events = df[df['EVENT_TYPE'].isin(flood_types)].copy()


    # Handle events with coordinates
    storm_events_geo = storm_events.dropna(subset=['BEGIN_LAT', 'BEGIN_LON']).copy()
    flood_events_geo = flood_events.dropna(subset=['BEGIN_LAT', 'BEGIN_LON']).copy()

    # Create geometries
    storm_geometry = [Point(xy) for xy in zip(storm_events_geo['BEGIN_LON'], storm_events_geo['BEGIN_LAT'])]
    flood_geometry = [Point(xy) for xy in zip(flood_events_geo['BEGIN_LON'], flood_events_geo['BEGIN_LAT'])]

    storm_gdf = gpd.GeoDataFrame(storm_events_geo, geometry=storm_geometry, crs='EPSG:4326')
    flood_gdf = gpd.GeoDataFrame(flood_events_geo, geometry=flood_geometry, crs='EPSG:4326')

    # Create a buffer around the input point
    center_point = Point(lon, lat)
    center_gdf = gpd.GeoDataFrame(index=[0], geometry=[center_point], crs='EPSG:4326')
    buffer = center_gdf.to_crs(epsg=3857).buffer(radius_km * 1000)
    buffer = buffer.to_crs(epsg=4326).unary_union
   
    # Find events within the buffer
    storms_nearby = storm_gdf[storm_gdf.geometry.within(buffer)]
    floods_nearby = flood_gdf[flood_gdf.geometry.within(buffer)]


    county_storm_events = storm_events.iloc[0:0].copy()
    county_flood_events = flood_events.iloc[0:0].copy()                     
    county_name, state_name = get_county_and_state(lat, lon)
    # Normalize state name
    #state_abbr = get_state_abbreviation(state_name)
    if county_name != "Unknown":
        
        county_name_norm = normalize_county_name(county_name)
        # Normalize 'CZ_NAME' in the dataset
        storm_events['CZ_NAME_NORM'] = storm_events['CZ_NAME'].str.upper().str.strip()
        flood_events['CZ_NAME_NORM'] = flood_events['CZ_NAME'].str.upper().str.strip()

        # Remove 'County' etc. from 'CZ_NAME_NORM'
        storm_events['CZ_NAME_NORM'] = storm_events['CZ_NAME_NORM'].apply(normalize_county_name)
        flood_events['CZ_NAME_NORM'] = flood_events['CZ_NAME_NORM'].apply(normalize_county_name)

        # Get events affecting the county
        county_storm_events = storm_events[
            (storm_events['CZ_NAME_NORM'] == county_name_norm) &
            (storm_events['CZ_TYPE'] == 'C') &
            (storm_events['STATE'] == state_name.upper().strip())
        ]

        county_flood_events = flood_events[
            (flood_events['CZ_NAME_NORM'] == county_name_norm) &
            (flood_events['CZ_TYPE'] == 'C') &
            (flood_events['STATE'] == state_name.upper().strip())
        ]
    
    # Combine events and remove duplicates
    storm_events_combined = pd.concat([storms_nearby, county_storm_events], ignore_index=True)
    flood_events_combined = pd.concat([floods_nearby, county_flood_events], ignore_index=True)

    # Remove duplicates based on 'EVENT_ID'
    storm_events_combined = storm_events_combined.drop_duplicates(subset='EVENT_ID')
    flood_events_combined = flood_events_combined.drop_duplicates(subset='EVENT_ID')
    
    # Return the counts
    return len(storm_events_combined), len(flood_events_combined)

def calculate_storm_score(storm_event_count, max_storm_event_count=45):
    if not storm_event_count:
        storm_event_count = 0
    normalized_storm_event_count = min(storm_event_count / max_storm_event_count, 1)
    storm_score = 1 - normalized_storm_event_count
    return storm_score

def calculate_flood_score(flood_event_count, max_flood_events=15):
    if not flood_event_count:
        flood_event_count = 0
    normalized_flood_events = min(flood_event_count / max_flood_events, 1)
    flood_score = 1 - normalized_flood_events
    return flood_score

In [49]:
input_lat = 41.681107
input_lon = -70.751257
user_input_state = "MA"

desired_zoom_factor = 0.0005
whether_find_loc = True
whether_get_score = True
whether_find_coord = False
find_radius = 0.02
find_step = 0.005
number_locations = 3
number_coordinates = 0

inundation_maps = state_inundation_maps.get(user_input_state)


def run_all():
    if not inundation_maps:
        no_result = print(f"The state {user_input_state} either has no sea level rise and coastal flood data, or you don't need to worry about it.")
        return no_result  # Exit the function if no inundation maps are available

    start_time = time.time()

#______________________________________________________________________
    # Process SSP585 Scenario
    result_ssp585, ssh_change_df_ssp585, highest_threshold_ssp585, next_output_ssp585, inundation_occurred_ssp585, inundation_year_ssp585, coordinates_ssp585, place_name_ssp585, next_inundation_status_ssp585 = process_scenario("SSP585", file_path_ar6_ssp585, inundation_maps, input_lat=input_lat, input_lon=input_lon)
    if result_ssp585 is None:
        bound_check = print("The data is unavailable or out of boundary")
        return bound_check  # Stop function execution here if no data is available
    print(result_ssp585)
    max_ssh_change_ssp585 = ssh_change_df_ssp585.loc[ssh_change_df_ssp585['Sea Level Rise'].idxmax()]
    print(next_output_ssp585)
    print(f"\nMaximum SLR is {max_ssh_change_ssp585['Sea Level Rise']} feet at the year {int(max_ssh_change_ssp585['Year'])}\n")
    if whether_find_loc:
        safe_location_ssp585, nearby_safe_locations_ssp585, nearby_safe_coordinates_ssp585, min_safe_distance_ssp585 = find_safe_locations(
            input_lat, input_lon, inundation_maps, inundation_occurred_ssp585, highest_threshold_ssp585, 
            find_radius=find_radius, find_step=find_step, max_location=number_locations, 
            max_coord=number_coordinates, whether_find_loc=whether_find_loc, whether_find_coord=whether_find_coord
        )
        print(safe_location_ssp585)
        print(f"\nNearby safe coordinates include ({nearby_safe_coordinates_ssp585})")


    # Process SSP245 Scenario
    result_ssp245, ssh_change_df_ssp245, highest_threshold_ssp245, next_output_ssp245, inundation_occurred_ssp245, inundation_year_ssp245, coordinates_ssp585, place_name_ssp585, next_inundation_status_ssp585 = process_scenario("SSP245", file_path_ar6_ssp245, inundation_maps, input_lat=input_lat, input_lon=input_lon)
    if result_ssp245 is None:
        bound_check = print("The data is unavailable or out of boundary")
        return bound_check  # Stop function execution here if no data is available
    print(result_ssp245)
    max_ssh_change_ssp245 = ssh_change_df_ssp245.loc[ssh_change_df_ssp245['Sea Level Rise'].idxmax()]
    print(next_output_ssp245)
    print(f"\nMaximum SLR is {max_ssh_change_ssp245['Sea Level Rise']} feet at the year {int(max_ssh_change_ssp245['Year'])}")
    if whether_find_loc or whether_find_coord:
        safe_location_ssp245, nearby_safe_locations_ssp245, nearby_safe_coordinates_ssp245, min_safe_distance_ssp245 = find_safe_locations(
            input_lat, input_lon, inundation_maps, inundation_occurred_ssp245, highest_threshold_ssp245, 
            find_radius=find_radius, find_step=find_step, max_location=number_locations, 
            max_coord=number_coordinates, whether_find_loc=whether_find_loc, whether_find_coord=whether_find_coord
        )
        print(safe_location_ssp245)
        print(f"\nNearby safe coordinates include ({nearby_safe_coordinates_ssp245})")

#______________________________________________________________________
    if whether_find_loc and whether_get_score:
        slr_score = calculate_slr_score(max_ssh_change_ssp585['Sea Level Rise'])
        inundation_score = calculate_inundation_score(inundation_year_ssp585)
        proximity_score = calculate_proximity_score(min_safe_distance_ssp585, inundation_year_ssp585)
        storm_count, flood_count = get_storm_and_flood_frequency(input_lat, input_lon, storm_and_flood_df)
        storm_score = calculate_storm_score(storm_count)
        flood_score = calculate_flood_score(flood_count)
        risk_assessment_score = calculate_risk_score(slr_score, inundation_score, proximity_score, storm_score, flood_score, weights)
        print(f"-----------------\n\nthe Risk Assessment Score (0 Most Dangerous, 100 Safest) for your locaton is: {round(risk_assessment_score)}")  
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"The function took {elapsed_time:.2f} seconds to run.")

run_all()

Threshold: 0 feet, Year Exceeded: 2020, Inundation: NoThreshold: 0.5 feet, Year Exceeded: 2030, Inundation: NoThreshold: 1 feet, Year Exceeded: 2050, Inundation: NoThreshold: 1.5 feet, Year Exceeded: 2070, Inundation: NoThreshold: 2 feet, Year Exceeded: 2080, Inundation: NoThreshold: 2.5 feet, Year Exceeded: 2090, Inundation: NoThreshold: 3 feet, Year Exceeded: 2100, Inundation: No
Next Unmet Threshold: 3.5 feet, Year Exceeded: Unknown, Inundation: No

Maximum SLR is 3.13238199 feet at the year 2100

Marion (coordinate: 41.676106999999995, -70.746257). Distance: 0.43 miles.Mattapoisett (coordinate: 41.671107, -70.761257). Distance: 0.86 miles.

Nearby safe coordinates include (None)
Threshold: 0 feet, Year Exceeded: 2020, Inundation: NoThreshold: 0.5 feet, Year Exceeded: 2030, Inundation: NoThreshold: 1 feet, Year Exceeded: 2050, Inundation: NoThreshold: 1.5 feet, Year Exceeded: 2070, Inundation: NoThreshold: 2 feet, Year Exceeded: 2090, Inundation: No
Next Unmet Threshold: 2.5 feet, Y

In [57]:
weights = {
    'slr': 0.1, #Maximum SSH of the ocean near the location
    'inundation': 0.5, #1, 0.5, 0, whether the inundation will be inundated
    'proximity_safe_location': 0.1, #distance between the closest safe neighborhood and your house. Change it to only when knowing that the house is inundated, only when inundation score yields 0.5 or 0.
    'storm': 0.15, #storm frequency
    'flood': 0.15 #historical flood count
}

blank_points = [
    (), (), (), (),
    (), (), (), (),
    (), (), (), (),
    (), (), (), (),
    (), (), (), (),
    (), (), (), (),
    (), (), (), ()
]

def find_many_points(points, scenario = "SSP585", file_path = file_path_ar6_ssp585, inundation_maps = None):
    slr_list = []
    inundation_list = []
    proximity_list = []
    storm_list = []
    flood_list = []
    risk_assessment_score = []
    for input_lat, input_lon in points:
        output, ssh_change_df, highest_threshold, next_output, inundation_occurred, inundation_year, coordinates, place_name, next_inundation_status = process_scenario(scenario, file_path, inundation_maps, input_lat=input_lat, input_lon=input_lon)
        inundation_list.append(inundation_occurred)
        if ssh_change_df is not None:
            max_ssh_change = ssh_change_df.loc[ssh_change_df['Sea Level Rise'].idxmax()]
            max_ssh_change_value = max_ssh_change['Sea Level Rise']
            slr_list.append(max_ssh_change['Sea Level Rise'])
        else:
            max_ssh_change = None
            max_ssh_change_value = None
            slr_list.append(max_ssh_change)
            
        safe_location, nearby_safe_locations, nearby_safe_coordinates, min_safe_distance = find_safe_locations(input_lat, input_lon, inundation_maps, 
                                                                                                               inundation_occurred, highest_threshold, 
                                                                                                               find_radius=0.02, find_step=0.005, max_location=3, 
                                                                                                               max_coord=0, whether_find_loc=True, whether_find_coord=False
                                                                                                              )
        proximity_list.append(min_safe_distance)
        storm_count, flood_count = get_storm_and_flood_frequency(input_lat, input_lon, storm_and_flood_df, radius_km=20)
        storm_list.append(storm_count)
        flood_list.append(flood_count)

        slr_score = calculate_slr_score(max_ssh_change_value, max_expected_slr=3.5)
        inundation_score = calculate_inundation_score(inundation_year)
        proximity_score = calculate_proximity_score(min_safe_distance, inundation_year, max_distance=2)
        storm_score = calculate_storm_score(storm_count, max_storm_event_count=45)
        flood_score = calculate_flood_score(flood_count, max_flood_events=15)
        risk_score = calculate_risk_score(slr_score, inundation_score, proximity_score, storm_score, flood_score, weights)
        risk_assessment_score.append(risk_score)
        
    return slr_list, inundation_list, proximity_list, storm_list, flood_list, risk_assessment_score


In [68]:
def calculate_year_score(scenario, file_path, inundation_maps, input_lat, input_lon):
    slr_df = get_sea_level_projection(input_lat, input_lon, file_path)
    mm_to_meters = 0.001
    meters_to_feet = 3.28084
    slr_df['Sea Level Rise'] = slr_df['Sea Level Rise'] * mm_to_meters * meters_to_feet
    
    slr_year_df = slr_df['Year'].tolist()
    slr_value_df = slr_df['Sea Level Rise'].tolist()
    slr_list = list(zip(slr_year_df, slr_value_df))

    available_thresholds = sorted(inundation_maps.keys())
    highest_threshold_minus = max(available_thresholds) - 1
    
    storm_count, flood_count = get_storm_and_flood_frequency(input_lat, input_lon, storm_and_flood_df, radius_km=20)
    storm_score = calculate_storm_score(storm_count, max_storm_event_count=45)
    flood_score = calculate_flood_score(flood_count, max_flood_events=15)

    safe_location, nearby_safe_locations, nearby_safe_coordinates, min_safe_distance = find_safe_locations(input_lat, input_lon, inundation_maps, 
                                                                                                               inundation_occurance=True, highest_threshold_exceeded=highest_threshold_minus, 
                                                                                                               find_radius=0.02, find_step=0.005, max_location=3, 
                                                                                                               max_coord=0, whether_find_loc=True, whether_find_coord=False
                                                                                                              )
    proximity_score = calculate_proximity_score(min_safe_distance, inundation_year=True, max_distance=2)
    slr_risk_scores = []
    inundation_risk_scores = []
    proximity_risk_scores = []
    flood_risk_scores = []
    storm_risk_scores = []
    year_risk_scores = []
    
    inundation_year_list = []
    for year,slr in slr_list:
        inundation_year = None
        slr_score = calculate_slr_score(slr, max_expected_slr=3.5)
        inundation_occurred = False
        for threshold in available_thresholds:
            if slr >= threshold:
                inundation_occurred = check_inundation(inundation_maps[threshold], input_lat, input_lon, window_size=10)
                if inundation_occurred:
                    break
                    
        if inundation_occurred:
            inundation_year = year
            inundation_year_list.append(inundation_year)
        if not inundation_occurred:
                proximity_score = 1
            
        # inundation_score = calculate_inundation_score(inundation_year)
        # if inundation_year:
        #     if any(yr < year for yr in inundation_year_list):
        #         if any(yr <= 2030 for yr in inundation_year_list):
        #             inundation_score = 0
        #         inundation_score = calculate_inundation_score(yr)

        first_inundation_year = min(inundation_year_list) if inundation_year_list else None

        inundation_score = calculate_inundation_score(first_inundation_year) if first_inundation_year else 1
                
        risk_score = calculate_risk_score(slr_score, inundation_score, proximity_score, storm_score, flood_score, weights)
        risk_score = round(risk_score)
        year_risk_scores.append(risk_score)
        storm_risk_scores.append(round(storm_score*100))
        flood_risk_scores.append(round(flood_score*100))
        slr_risk_scores.append(round(slr_score*100))
        inundation_risk_scores.append(round(inundation_score*100))
        proximity_risk_scores.append(round(proximity_score*100))
    return year_risk_scores, slr_risk_scores, inundation_risk_scores, proximity_risk_scores, storm_risk_scores, flood_risk_scores

In [77]:
def calculate_year_scores_for_all_points(points, scenario, file_path, inundation_maps):
    # List to store the year scores for each point
    all_year_scores = []
    all_slr_scores = []
    all_inundation_scores = []
    all_proximity_scores = []
    all_storm_scores = []
    all_flood_scores = []
    # Loop through each point (lat, lon) in the points list
    for idx, (input_lat, input_lon) in enumerate(points, start=1):
        # Get year scores for the current point
        year_scores, slr_scores, inundation_scores, proximity_scores, storm_scores, flood_scores = calculate_year_score(scenario, file_path, inundation_maps, input_lat, input_lon)
        
        # Append the year scores to the list
        all_year_scores.append(year_scores)
        all_slr_scores.append(slr_scores)
        all_inundation_scores.append(inundation_scores)
        all_proximity_scores.append(proximity_scores)
        all_storm_scores.append(storm_scores)
        all_flood_scores.append(flood_scores)
    # Define the years corresponding to the scores (2020 to 2100)
    years = [2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100]
    
    # Create a DataFrame with each row representing the scores for a point and columns as years
    df_year_scores = pd.DataFrame(all_year_scores, columns=years)
    df_slr_scores = pd.DataFrame(all_slr_scores, columns=years)
    df_inundation_scores = pd.DataFrame(all_inundation_scores, columns=years)
    df_proximity_scores = pd.DataFrame(all_proximity_scores, columns=years)
    df_storm_scores = pd.DataFrame(all_storm_scores, columns=years)
    df_flood_scores = pd.DataFrame(all_flood_scores, columns=years)
    return df_year_scores, df_slr_scores, df_inundation_scores, df_proximity_scores, df_storm_scores, df_flood_scores

In [79]:
MA_points = [(41.560607, -70.936777), (41.565849, -70.939113), (41.581573, -70.943054), (41.596859, -70.928495), 
            (41.606234, -70.904464), (41.608318, -70.864587), (41.653431, -70.792897), (41.681019, -70.758942),
             (41.715625, -70.614196), (41.748468, -70.436862), (42.010510, -70.082880), (41.924087, -70.544832),
             (42.249851, -70.979433), (42.356573, -71.017367), (42.468619, -70.900101), (42.937922, -70.801979),
             (42.557918, -70.817436), (41.588883, -70.642907), (41.574060, -70.464419), (41.844237, -70.001492),
             (42.106916, -70.664196), (42.315835, -71.033523), (42.341551, -71.018082), (42.368306, -71.055174),
            (42.382859, -70.973359), (42.415886, -70.989154), (42.276716, -71.011516), (42.299167, -71.026601)
            ]
points = MA_points
inundation_maps = inundation_maps_MA
df_year_scores_MA_585, df_slr_scores_MA_585, df_inundation_scores_MA_585, df_proximity_scores_MA_585, df_storm_scores_MA_585, df_flood_scores_MA_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_MA_245, df_slr_scores_MA_245, df_inundation_scores_MA_245, df_proximity_scores_MA_245, df_storm_scores_MA_245, df_flood_scores_MA_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


FL_points = [
    (25.710288, -80.248741), (25.711359, -80.249031), (25.713607, -80.249205), (25.716430, -80.246159),
    (25.722651, -80.241691), (25.727879, -80.233596), (25.731930, -80.232030), (25.734883, -80.226169),
    (25.739692, -80.217610), (25.735458, -80.219815), (25.750197, -80.200058), (25.756207, -80.192631),
    (25.764020, -80.188511), (25.769376, -80.183724), (25.773021, -80.174953), (25.769636, -80.166171),
    (25.789411, -80.185508), (25.787964, -80.185874), (25.767063, -80.145641), (25.801229, -80.185612),
    (25.798117, -80.185488), (25.750499, -80.146885), (25.808858, -80.185458), (25.814029, -80.185438),
    (25.828122, -80.180148), (25.829501, -80.180148), (25.831493, -80.180537), (25.843260, -80.174125)
]
points = FL_points
inundation_maps = inundation_maps_FL
df_year_scores_FL_585, df_slr_scores_FL_585, df_inundation_scores_FL_585, df_proximity_scores_FL_585, df_storm_scores_FL_585, df_flood_scores_FL_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_FL_245, df_slr_scores_FL_245, df_inundation_scores_FL_245, df_proximity_scores_FL_245, df_storm_scores_FL_245, df_flood_scores_FL_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


CA_points = [
    (33.995670, -118.481175), (33.994332, -118.479910), (33.992260, -118.478106), (33.984547, -118.471431),
    (33.983441, -118.470589), (33.980933, -118.468220), (33.978273, -118.465994), (33.966969, -118.453742),
    (34.027915, -118.518500), (34.027003, -118.516969), (34.038811, -118.554159), (34.039654, -118.558538),
    (34.041683, -118.564441), (34.042483, -118.568966), (34.041887, -118.576805), (34.039840, -118.549802),
    (34.000963, -118.484727), (34.025433, -118.512226), (34.024745, -118.510618), (34.022617, -118.507740),
    (34.016728, -118.500791), (33.997855, -118.482665), (33.713437, -118.310474), (33.712465, -118.307591),
    (33.707400, -118.286125), (33.717434, -118.257453), (33.727250, -118.267017), (33.740619, -118.252977)
]
points = CA_points
inundation_maps = inundation_maps_CA
df_year_scores_CA_585, df_slr_scores_CA_585, df_inundation_scores_CA_585, df_proximity_scores_CA_585, df_storm_scores_CA_585, df_flood_scores_CA_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_CA_245, df_slr_scores_CA_245, df_inundation_scores_CA_245, df_proximity_scores_CA_245, df_storm_scores_CA_245, df_flood_scores_CA_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


NY_points = [
    (40.895381, -73.914268), (40.889338, -73.916972), (40.872985, -73.826694), (40.853694, -73.937895),
    (40.870052, -73.893814), (40.820718, -73.907498), (40.785664, -73.948408), (40.761136, -73.985058),
    (40.735954, -73.986478), (40.720238, -73.980796), (40.779210, -73.816869), (40.885186, -73.614587),
    (40.584352, -73.935881), (40.599723, -74.003273), (40.606774, -74.028992), (40.634430, -74.036374),
    (40.678190, -74.017009), (40.730229, -74.009150), (40.710025, -73.983097), (40.703136, -74.011422),
    (40.739407, -74.011275), (40.765401, -73.997449), (40.774759, -73.944583), (40.803061, -73.970547),
    (40.701001, -73.972433), (40.574303, -74.006938), (40.805118, -73.885627), (40.808528, -73.857165),
    (40.799545, -73.911341), (40.795670, -73.800737), (40.777185, -73.768336), (40.862436, -73.694524),
    (40.612986, -73.766800), (40.617175, -73.755617), (40.627645, -73.748375), (40.641665, -73.722265),
    (40.877230, -73.653813), (40.824890, -73.701513), (40.628445, -73.595870), (40.592314, -73.674647)   
]
points = NY_points
inundation_maps = inundation_maps_NY
df_year_scores_NY_585, df_slr_scores_NY_585, df_inundation_scores_NY_585, df_proximity_scores_NY_585, df_storm_scores_NY_585, df_flood_scores_NY_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_NY_245, df_slr_scores_NY_245, df_inundation_scores_NY_245, df_proximity_scores_NY_245, df_storm_scores_NY_245, df_flood_scores_NY_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


PA_points = [(39.815000, -75.413719),(39.828980, -75.384748),(39.978452, -75.097862),(39.938166, -75.143754),
            (39.920865, -75.138882),(39.890382, -75.146830),(39.966467, -75.132473),(40.003790, -75.059918),
            (40.047085, -74.983793),(40.061286, -74.960407),(40.066417, -74.945752),(40.077320, -74.897452),
            (40.086982, -74.862997),(40.105944, -74.838364),(40.125856, -74.829165),(40.126928, -74.778964),
            (40.200917, -74.766179),(40.228537, -74.798452),(40.359194, -74.949356),(39.945794, -75.141224),
            (39.937123, -75.142637),(39.953180, -75.140805),(39.967396, -75.125977),(39.960803, -75.137608)]
points = PA_points
inundation_maps = inundation_maps_PA
df_year_scores_PA_585, df_slr_scores_PA_585, df_inundation_scores_PA_585, df_proximity_scores_PA_585, df_storm_scores_PA_585, df_flood_scores_PA_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_PA_245, df_slr_scores_PA_245, df_inundation_scores_PA_245, df_proximity_scores_PA_245, df_storm_scores_PA_245, df_flood_scores_PA_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


BOS_points = [
    (42.331558, -71.078278), (42.340241, -71.052201), (42.378432, -71.090024), (42.378825, -71.105728),
    (42.357453, -71.105043), (42.358496, -71.101579), (42.364750, -71.093306), (42.358401, -71.064636),
    (42.382378, -71.062199), (42.372239, -70.988825), (42.377925, -71.051167), (42.358780, -71.114792),
    (42.388091, -71.074312), (42.349515, -71.113738), (42.354634, -71.102550), (42.353059, -71.047851),
    (42.322471, -71.047140), (42.354240, -71.075556), (42.385992, -71.072359), (42.390452, -71.066853),
    (42.371430, -71.110364), (42.415521, -71.049916), (42.348700, -71.116093), (42.366163, -71.077556),
    (42.385280, -71.005802), (42.399262, -71.087397), (42.341084, -71.025091), (42.359990, -70.973449)
]
points = BOS_points
inundation_maps = inundation_maps_MA
df_year_scores_BOS_585, df_slr_scores_BOS_585, df_inundation_scores_BOS_585, df_proximity_scores_BOS_585, df_storm_scores_BOS_585, df_flood_scores_BOS_585 = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_BOS_245, df_slr_scores_BOS_245, df_inundation_scores_BOS_245, df_proximity_scores_BOS_245, df_storm_scores_BOS_245, df_flood_scores_BOS_245 = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)

In [80]:
PA_points_new = [
    (39.936491, -75.151825), (39.923660, -75.142007), (39.963039, -75.136260), (39.971029, -75.118018),
    (39.961791, -75.144567), (39.932538, -75.161290), (39.980130, -75.143522), (39.939115, -75.152917),
    (39.921575, -75.179059), (39.967989, -75.215491), (40.006568, -75.069793), (39.997130, -75.068721),
    (39.913367, -75.137821), (40.022567, -75.047831), (40.073413, -74.980874), (40.004106, -75.064436)
]
points = PA_points_new
inundation_maps = inundation_maps_PA
df_year_scores_PA_585_new, df_slr_scores_PA_585_new, df_inundation_scores_PA_585_new, df_proximity_scores_PA_585_new, df_storm_scores_PA_585_new, df_flood_scores_PA_585_new = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_PA_245_new, df_slr_scores_PA_245_new, df_inundation_scores_PA_245_new, df_proximity_scores_PA_245_new, df_storm_scores_PA_245_new, df_flood_scores_PA_245_new = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


MA_points_new = [
    (42.344192, -71.075606), (42.340543, -71.051049), (42.368231, -71.081049), (42.351021, -71.085352),
    (42.380976, -71.067039), (42.380976, -71.037028), (42.323180, -71.074242), (42.284065, -71.130862),
    (42.362730, -71.125262), (42.350314, -71.10141), (42.352079, -71.083222), (42.377704, -71.127219)
]
points = MA_points_new
inundation_maps = inundation_maps_MA
df_year_scores_MA_585_new, df_slr_scores_MA_585_new, df_inundation_scores_MA_585_new, df_proximity_scores_MA_585_new, df_storm_scores_MA_585_new, df_flood_scores_MA_585_new = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_MA_245_new, df_slr_scores_MA_245_new, df_inundation_scores_MA_245_new, df_proximity_scores_MA_245_new, df_storm_scores_MA_245_new, df_flood_scores_MA_245_new = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


BOS_points_new = [
    (42.344192, -71.075606), (42.340543, -71.051049), (42.368231, -71.081049), (42.351021, -71.085352),
    (42.380976, -71.067039), (42.380976, -71.037028), (42.323180, -71.074242), (42.284065, -71.130862),
    (42.362730, -71.125262), (42.350314, -71.10141), (42.352079, -71.083222), (42.377704, -71.127219)
]
df_year_scores_BOS_585_new, df_slr_scores_BOS_585_new, df_inundation_scores_BOS_585_new, df_proximity_scores_BOS_585_new, df_storm_scores_BOS_585_new, df_flood_scores_BOS_585_new = df_year_scores_MA_585_new.copy(), df_slr_scores_MA_585_new.copy(), df_inundation_scores_MA_585_new.copy(), df_proximity_scores_MA_585_new.copy(), df_storm_scores_MA_585_new.copy(), df_flood_scores_MA_585_new.copy()
df_year_scores_BOS_245_new, df_slr_scores_BOS_245_new, df_inundation_scores_BOS_245_new, df_proximity_scores_BOS_245_new, df_storm_scores_BOS_245_new, df_flood_scores_BOS_245_new = df_year_scores_MA_245_new.copy(), df_slr_scores_MA_245_new.copy(), df_inundation_scores_MA_245_new.copy(), df_proximity_scores_MA_245_new.copy(), df_storm_scores_MA_245_new.copy(), df_flood_scores_MA_245_new.copy()


FL_points_new = [
    (25.827304, -80.181043), (25.830703, -80.180247), (25.832638, -80.181097), (25.842615, -80.176481),
    (25.854386, -80.176362), (25.830564, -80.186898), (25.817687, -80.191609), (25.816608, -80.185784),
    (25.828559, -80.182444), (25.827689, -80.180030), (25.832035, -80.180866), (25.820023, -80.179497)
]
points = FL_points_new
inundation_maps = inundation_maps_FL
df_year_scores_FL_585_new, df_slr_scores_FL_585_new, df_inundation_scores_FL_585_new, df_proximity_scores_FL_585_new, df_storm_scores_FL_585_new, df_flood_scores_FL_585_new = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_FL_245_new, df_slr_scores_FL_245_new, df_inundation_scores_FL_245_new, df_proximity_scores_FL_245_new, df_storm_scores_FL_245_new, df_flood_scores_FL_245_new = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)


CA_points_new = [
    (33.733449, -118.256768), (33.779885, -118.272089), (34.067814, -118.337867), (34.011927, -118.300462),
    (34.204323, -118.577602), (34.032126, -118.436482), (34.008169, -118.444983), (33.960956, -118.434747),
    (33.955669, -118.443033), (33.962542, -118.430923), (34.061341, -118.437297), (34.032823, -118.518881)
]
points = CA_points_new
inundation_maps = inundation_maps_CA
df_year_scores_CA_585_new, df_slr_scores_CA_585_new, df_inundation_scores_CA_585_new, df_proximity_scores_CA_585_new, df_storm_scores_CA_585_new, df_flood_scores_CA_585_new = calculate_year_scores_for_all_points(points, 'SSP585', file_path_ar6_ssp585, inundation_maps)
df_year_scores_CA_245_new, df_slr_scores_CA_245_new, df_inundation_scores_CA_245_new, df_proximity_scores_CA_245_new, df_storm_scores_CA_245_new, df_flood_scores_CA_245_new = calculate_year_scores_for_all_points(points, 'SSP245', file_path_ar6_ssp245, inundation_maps)

In [81]:
# df_year_scores_MA_585['State'] = 'MA'
# df_year_scores_MA_585['Scenario'] = 'SSP585'
# df_year_scores_MA_585_new['State'] = 'MA'
# df_year_scores_MA_585_new['Scenario'] = 'SSP585'

# df_year_scores_MA_245['State'] = 'MA'
# df_year_scores_MA_245['Scenario'] = 'SSP245'
# df_year_scores_MA_245_new['State'] = 'MA'
# df_year_scores_MA_245_new['Scenario'] = 'SSP245'

# df_year_scores_PA_585['State'] = 'PA'
# df_year_scores_PA_585['Scenario'] = 'SSP585'
# df_year_scores_PA_585_new['State'] = 'PA'
# df_year_scores_PA_585_new['Scenario'] = 'SSP585'

# df_year_scores_PA_245['State'] = 'PA'
# df_year_scores_PA_245['Scenario'] = 'SSP245'
# df_year_scores_PA_245_new['State'] = 'PA'
# df_year_scores_PA_245_new['Scenario'] = 'SSP245'

# df_year_scores_FL_585['State'] = 'FL'
# df_year_scores_FL_585['Scenario'] = 'SSP585'
# df_year_scores_FL_585_new['State'] = 'FL'
# df_year_scores_FL_585_new['Scenario'] = 'SSP585'

# df_year_scores_FL_245['State'] = 'FL'
# df_year_scores_FL_245['Scenario'] = 'SSP245'
# df_year_scores_FL_245_new['State'] = 'FL'
# df_year_scores_FL_245_new['Scenario'] = 'SSP245'

# df_year_scores_CA_585['State'] = 'CA'
# df_year_scores_CA_585['Scenario'] = 'SSP585'
# df_year_scores_CA_585_new['State'] = 'CA'
# df_year_scores_CA_585_new['Scenario'] = 'SSP585'

# df_year_scores_CA_245['State'] = 'CA'
# df_year_scores_CA_245['Scenario'] = 'SSP245'
# df_year_scores_CA_245_new['State'] = 'CA'
# df_year_scores_CA_245_new['Scenario'] = 'SSP245'

# df_year_scores_NY_585['State'] = 'NY'
# df_year_scores_NY_585['Scenario'] = 'SSP585'

# df_year_scores_NY_245['State'] = 'NY'
# df_year_scores_NY_245['Scenario'] = 'SSP245'

# df_year_scores_BOS_585['State'] = 'BOS'
# df_year_scores_BOS_585['Scenario'] = 'SSP585'
# df_year_scores_BOS_585_new['State'] = 'BOS'
# df_year_scores_BOS_585_new['Scenario'] = 'SSP585'

# df_year_scores_BOS_245['State'] = 'BOS'
# df_year_scores_BOS_245['Scenario'] = 'SSP245'
# df_year_scores_BOS_245_new['State'] = 'BOS'
# df_year_scores_BOS_245_new['Scenario'] = 'SSP245'


# # Concatenate all DataFrames together
# combined_df = pd.concat([
#     df_year_scores_MA_585, df_year_scores_MA_585_new, df_year_scores_MA_245, df_year_scores_MA_245_new,
#     df_year_scores_PA_585, df_year_scores_PA_585_new, df_year_scores_PA_245, df_year_scores_PA_245_new,
#     df_year_scores_FL_585, df_year_scores_FL_585_new, df_year_scores_FL_245, df_year_scores_FL_245_new,
#     df_year_scores_CA_585, df_year_scores_CA_585_new, df_year_scores_CA_245, df_year_scores_CA_245_new,
#     df_year_scores_NY_585, df_year_scores_NY_245,
#     df_year_scores_BOS_585, df_year_scores_BOS_585_new, df_year_scores_BOS_245, df_year_scores_BOS_245_new,
# ], ignore_index=True)

# # Export the combined DataFrame to an Excel file
# combined_df.to_excel('combined_year_scores_new.xlsx', index=False)
# combined_df.to_csv('combined_year_scores_new.csv', index=False)

In [82]:
import pandas as pd

# Add 'State' and 'Scenario' columns for each score type
# Year Scores
for df, state, scenario in [
    (df_year_scores_MA_585, 'MA', 'SSP585'), (df_year_scores_MA_585_new, 'MA', 'SSP585'),
    (df_year_scores_MA_245, 'MA', 'SSP245'), (df_year_scores_MA_245_new, 'MA', 'SSP245'),
    (df_year_scores_PA_585, 'PA', 'SSP585'), (df_year_scores_PA_585_new, 'PA', 'SSP585'),
    (df_year_scores_PA_245, 'PA', 'SSP245'), (df_year_scores_PA_245_new, 'PA', 'SSP245'),
    (df_year_scores_FL_585, 'FL', 'SSP585'), (df_year_scores_FL_585_new, 'FL', 'SSP585'),
    (df_year_scores_FL_245, 'FL', 'SSP245'), (df_year_scores_FL_245_new, 'FL', 'SSP245'),
    (df_year_scores_CA_585, 'CA', 'SSP585'), (df_year_scores_CA_585_new, 'CA', 'SSP585'),
    (df_year_scores_CA_245, 'CA', 'SSP245'), (df_year_scores_CA_245_new, 'CA', 'SSP245'),
    (df_year_scores_NY_585, 'NY', 'SSP585'), (df_year_scores_NY_245, 'NY', 'SSP245'),
    (df_year_scores_BOS_585, 'BOS', 'SSP585'), (df_year_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_year_scores_BOS_245, 'BOS', 'SSP245'), (df_year_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# SLR Scores
for df, state, scenario in [
    (df_slr_scores_MA_585, 'MA', 'SSP585'), (df_slr_scores_MA_585_new, 'MA', 'SSP585'),
    (df_slr_scores_MA_245, 'MA', 'SSP245'), (df_slr_scores_MA_245_new, 'MA', 'SSP245'),
    (df_slr_scores_PA_585, 'PA', 'SSP585'), (df_slr_scores_PA_585_new, 'PA', 'SSP585'),
    (df_slr_scores_PA_245, 'PA', 'SSP245'), (df_slr_scores_PA_245_new, 'PA', 'SSP245'),
    (df_slr_scores_FL_585, 'FL', 'SSP585'), (df_slr_scores_FL_585_new, 'FL', 'SSP585'),
    (df_slr_scores_FL_245, 'FL', 'SSP245'), (df_slr_scores_FL_245_new, 'FL', 'SSP245'),
    (df_slr_scores_CA_585, 'CA', 'SSP585'), (df_slr_scores_CA_585_new, 'CA', 'SSP585'),
    (df_slr_scores_CA_245, 'CA', 'SSP245'), (df_slr_scores_CA_245_new, 'CA', 'SSP245'),
    (df_slr_scores_NY_585, 'NY', 'SSP585'), (df_slr_scores_NY_245, 'NY', 'SSP245'),
    (df_slr_scores_BOS_585, 'BOS', 'SSP585'), (df_slr_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_slr_scores_BOS_245, 'BOS', 'SSP245'), (df_slr_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# Inundation Scores
for df, state, scenario in [
    (df_inundation_scores_MA_585, 'MA', 'SSP585'), (df_inundation_scores_MA_585_new, 'MA', 'SSP585'),
    (df_inundation_scores_MA_245, 'MA', 'SSP245'), (df_inundation_scores_MA_245_new, 'MA', 'SSP245'),
    (df_inundation_scores_PA_585, 'PA', 'SSP585'), (df_inundation_scores_PA_585_new, 'PA', 'SSP585'),
    (df_inundation_scores_PA_245, 'PA', 'SSP245'), (df_inundation_scores_PA_245_new, 'PA', 'SSP245'),
    (df_inundation_scores_FL_585, 'FL', 'SSP585'), (df_inundation_scores_FL_585_new, 'FL', 'SSP585'),
    (df_inundation_scores_FL_245, 'FL', 'SSP245'), (df_inundation_scores_FL_245_new, 'FL', 'SSP245'),
    (df_inundation_scores_CA_585, 'CA', 'SSP585'), (df_inundation_scores_CA_585_new, 'CA', 'SSP585'),
    (df_inundation_scores_CA_245, 'CA', 'SSP245'), (df_inundation_scores_CA_245_new, 'CA', 'SSP245'),
    (df_inundation_scores_NY_585, 'NY', 'SSP585'), (df_inundation_scores_NY_245, 'NY', 'SSP245'),
    (df_inundation_scores_BOS_585, 'BOS', 'SSP585'), (df_inundation_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_inundation_scores_BOS_245, 'BOS', 'SSP245'), (df_inundation_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# Repeat the same process for Proximity, Storm, and Flood Scores
# Proximity Scores
for df, state, scenario in [
    (df_proximity_scores_MA_585, 'MA', 'SSP585'), (df_proximity_scores_MA_585_new, 'MA', 'SSP585'),
    (df_proximity_scores_MA_245, 'MA', 'SSP245'), (df_proximity_scores_MA_245_new, 'MA', 'SSP245'),
    (df_proximity_scores_PA_585, 'PA', 'SSP585'), (df_proximity_scores_PA_585_new, 'PA', 'SSP585'),
    (df_proximity_scores_PA_245, 'PA', 'SSP245'), (df_proximity_scores_PA_245_new, 'PA', 'SSP245'),
    (df_proximity_scores_FL_585, 'FL', 'SSP585'), (df_proximity_scores_FL_585_new, 'FL', 'SSP585'),
    (df_proximity_scores_FL_245, 'FL', 'SSP245'), (df_proximity_scores_FL_245_new, 'FL', 'SSP245'),
    (df_proximity_scores_CA_585, 'CA', 'SSP585'), (df_proximity_scores_CA_585_new, 'CA', 'SSP585'),
    (df_proximity_scores_CA_245, 'CA', 'SSP245'), (df_proximity_scores_CA_245_new, 'CA', 'SSP245'),
    (df_proximity_scores_NY_585, 'NY', 'SSP585'), (df_proximity_scores_NY_245, 'NY', 'SSP245'),
    (df_proximity_scores_BOS_585, 'BOS', 'SSP585'), (df_proximity_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_proximity_scores_BOS_245, 'BOS', 'SSP245'), (df_proximity_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# Storm Scores
for df, state, scenario in [
    (df_storm_scores_MA_585, 'MA', 'SSP585'), (df_storm_scores_MA_585_new, 'MA', 'SSP585'),
    (df_storm_scores_MA_245, 'MA', 'SSP245'), (df_storm_scores_MA_245_new, 'MA', 'SSP245'),
    (df_storm_scores_PA_585, 'PA', 'SSP585'), (df_storm_scores_PA_585_new, 'PA', 'SSP585'),
    (df_storm_scores_PA_245, 'PA', 'SSP245'), (df_storm_scores_PA_245_new, 'PA', 'SSP245'),
    (df_storm_scores_FL_585, 'FL', 'SSP585'), (df_storm_scores_FL_585_new, 'FL', 'SSP585'),
    (df_storm_scores_FL_245, 'FL', 'SSP245'), (df_storm_scores_FL_245_new, 'FL', 'SSP245'),
    (df_storm_scores_CA_585, 'CA', 'SSP585'), (df_storm_scores_CA_585_new, 'CA', 'SSP585'),
    (df_storm_scores_CA_245, 'CA', 'SSP245'), (df_storm_scores_CA_245_new, 'CA', 'SSP245'),
    (df_storm_scores_NY_585, 'NY', 'SSP585'), (df_storm_scores_NY_245, 'NY', 'SSP245'),
    (df_storm_scores_BOS_585, 'BOS', 'SSP585'), (df_storm_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_storm_scores_BOS_245, 'BOS', 'SSP245'), (df_storm_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# Flood Scores
for df, state, scenario in [
    (df_flood_scores_MA_585, 'MA', 'SSP585'), (df_flood_scores_MA_585_new, 'MA', 'SSP585'),
    (df_flood_scores_MA_245, 'MA', 'SSP245'), (df_flood_scores_MA_245_new, 'MA', 'SSP245'),
    (df_flood_scores_PA_585, 'PA', 'SSP585'), (df_flood_scores_PA_585_new, 'PA', 'SSP585'),
    (df_flood_scores_PA_245, 'PA', 'SSP245'), (df_flood_scores_PA_245_new, 'PA', 'SSP245'),
    (df_flood_scores_FL_585, 'FL', 'SSP585'), (df_flood_scores_FL_585_new, 'FL', 'SSP585'),
    (df_flood_scores_FL_245, 'FL', 'SSP245'), (df_flood_scores_FL_245_new, 'FL', 'SSP245'),
    (df_flood_scores_CA_585, 'CA', 'SSP585'), (df_flood_scores_CA_585_new, 'CA', 'SSP585'),
    (df_flood_scores_CA_245, 'CA', 'SSP245'), (df_flood_scores_CA_245_new, 'CA', 'SSP245'),
    (df_flood_scores_NY_585, 'NY', 'SSP585'), (df_flood_scores_NY_245, 'NY', 'SSP245'),
    (df_flood_scores_BOS_585, 'BOS', 'SSP585'), (df_flood_scores_BOS_585_new, 'BOS', 'SSP585'),
    (df_flood_scores_BOS_245, 'BOS', 'SSP245'), (df_flood_scores_BOS_245_new, 'BOS', 'SSP245')
]:
    df['State'] = state
    df['Scenario'] = scenario

# Combine DataFrames for each score type

# Year Scores
combined_year_scores = pd.concat([
    df_year_scores_MA_585, df_year_scores_MA_585_new, df_year_scores_MA_245, df_year_scores_MA_245_new,
    df_year_scores_PA_585, df_year_scores_PA_585_new, df_year_scores_PA_245, df_year_scores_PA_245_new,
    df_year_scores_FL_585, df_year_scores_FL_585_new, df_year_scores_FL_245, df_year_scores_FL_245_new,
    df_year_scores_CA_585, df_year_scores_CA_585_new, df_year_scores_CA_245, df_year_scores_CA_245_new,
    df_year_scores_NY_585, df_year_scores_NY_245,
    df_year_scores_BOS_585, df_year_scores_BOS_585_new, df_year_scores_BOS_245, df_year_scores_BOS_245_new
], ignore_index=True)

# SLR Scores
combined_slr_scores = pd.concat([
    df_slr_scores_MA_585, df_slr_scores_MA_585_new, df_slr_scores_MA_245, df_slr_scores_MA_245_new,
    df_slr_scores_PA_585, df_slr_scores_PA_585_new, df_slr_scores_PA_245, df_slr_scores_PA_245_new,
    df_slr_scores_FL_585, df_slr_scores_FL_585_new, df_slr_scores_FL_245, df_slr_scores_FL_245_new,
    df_slr_scores_CA_585, df_slr_scores_CA_585_new, df_slr_scores_CA_245, df_slr_scores_CA_245_new,
    df_slr_scores_NY_585, df_slr_scores_NY_245,
    df_slr_scores_BOS_585, df_slr_scores_BOS_585_new, df_slr_scores_BOS_245, df_slr_scores_BOS_245_new
], ignore_index=True)

# Inundation Scores
combined_inundation_scores = pd.concat([
    df_inundation_scores_MA_585, df_inundation_scores_MA_585_new, df_inundation_scores_MA_245, df_inundation_scores_MA_245_new,
    df_inundation_scores_PA_585, df_inundation_scores_PA_585_new, df_inundation_scores_PA_245, df_inundation_scores_PA_245_new,
    df_inundation_scores_FL_585, df_inundation_scores_FL_585_new, df_inundation_scores_FL_245, df_inundation_scores_FL_245_new,
    df_inundation_scores_CA_585, df_inundation_scores_CA_585_new, df_inundation_scores_CA_245, df_inundation_scores_CA_245_new,
    df_inundation_scores_NY_585, df_inundation_scores_NY_245,
    df_inundation_scores_BOS_585, df_inundation_scores_BOS_585_new, df_inundation_scores_BOS_245, df_inundation_scores_BOS_245_new
], ignore_index=True)

# Proximity Scores
combined_proximity_scores = pd.concat([
    df_proximity_scores_MA_585, df_proximity_scores_MA_585_new, df_proximity_scores_MA_245, df_proximity_scores_MA_245_new,
    df_proximity_scores_PA_585, df_proximity_scores_PA_585_new, df_proximity_scores_PA_245, df_proximity_scores_PA_245_new,
    df_proximity_scores_FL_585, df_proximity_scores_FL_585_new, df_proximity_scores_FL_245, df_proximity_scores_FL_245_new,
    df_proximity_scores_CA_585, df_proximity_scores_CA_585_new, df_proximity_scores_CA_245, df_proximity_scores_CA_245_new,
    df_proximity_scores_NY_585, df_proximity_scores_NY_245,
    df_proximity_scores_BOS_585, df_proximity_scores_BOS_585_new, df_proximity_scores_BOS_245, df_proximity_scores_BOS_245_new
], ignore_index=True)

# Storm Scores
combined_storm_scores = pd.concat([
    df_storm_scores_MA_585, df_storm_scores_MA_585_new, df_storm_scores_MA_245, df_storm_scores_MA_245_new,
    df_storm_scores_PA_585, df_storm_scores_PA_585_new, df_storm_scores_PA_245, df_storm_scores_PA_245_new,
    df_storm_scores_FL_585, df_storm_scores_FL_585_new, df_storm_scores_FL_245, df_storm_scores_FL_245_new,
    df_storm_scores_CA_585, df_storm_scores_CA_585_new, df_storm_scores_CA_245, df_storm_scores_CA_245_new,
    df_storm_scores_NY_585, df_storm_scores_NY_245,
    df_storm_scores_BOS_585, df_storm_scores_BOS_585_new, df_storm_scores_BOS_245, df_storm_scores_BOS_245_new
], ignore_index=True)

# Flood Scores
combined_flood_scores = pd.concat([
    df_flood_scores_MA_585, df_flood_scores_MA_585_new, df_flood_scores_MA_245, df_flood_scores_MA_245_new,
    df_flood_scores_PA_585, df_flood_scores_PA_585_new, df_flood_scores_PA_245, df_flood_scores_PA_245_new,
    df_flood_scores_FL_585, df_flood_scores_FL_585_new, df_flood_scores_FL_245, df_flood_scores_FL_245_new,
    df_flood_scores_CA_585, df_flood_scores_CA_585_new, df_flood_scores_CA_245, df_flood_scores_CA_245_new,
    df_flood_scores_NY_585, df_flood_scores_NY_245,
    df_flood_scores_BOS_585, df_flood_scores_BOS_585_new, df_flood_scores_BOS_245, df_flood_scores_BOS_245_new
], ignore_index=True)

# Export each score type to a separate sheet in an Excel workbook
with pd.ExcelWriter('combined_scores_new.xlsx') as writer:
    combined_year_scores.to_excel(writer, sheet_name='Year Scores', index=False)
    combined_slr_scores.to_excel(writer, sheet_name='SLR Scores', index=False)
    combined_inundation_scores.to_excel(writer, sheet_name='Inundation Scores', index=False)
    combined_proximity_scores.to_excel(writer, sheet_name='Proximity Scores', index=False)
    combined_storm_scores.to_excel(writer, sheet_name='Storm Scores', index=False)
    combined_flood_scores.to_excel(writer, sheet_name='Flood Scores', index=False)

print("Data exported to 'combined_scores_new.xlsx'")


Data exported to 'combined_scores_new.xlsx'
